In [ ]:
%%capture
!pip install matplotlib pandas numpy  scikit-learn catboost dvc mlflow
import warnings
warnings.simplefilter("ignore")
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as lir
from sklearn.model_selection import GridSearchCV
import pickle
import dvc.api
import mlflow
import mlflow.sklearn


%pylab inline

### Создаём новый Git-репозиторий


### Загружаем датасет, который будем использовать для модели,


In [ ]:
%%sh
git init && git checkout -b master
git config --global user.email "publicemail@example.com" && git config --global user.name "publicname"
dvc config core.analytics false
dvc init -f

Initialized empty Git repository in /content/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Switched to a new branch 'master'
ERROR: configuration error - config file error: Not inside a DVC repo


In [ ]:
df = pd.read_csv("https://gitlab.com//-/raw/main/energydata_complete.csv?ref_type=heads&inline=false")
df.dropna(inplace=True)
df.to_csv("energydata_complete_v1.csv")

In [ ]:
%%sh
dvc remote add -d myremote datasets -f
dvc add energydata_complete_v1.csv && dvc commit
git add energydata_complete_v1.csv.dvc .dvc/config && git commit -m ”initial_data”
dvc push

In [ ]:
x = df.drop(['Appliances','date','Windspeed','Visibility','Tdewpoint','rv1','rv2'],axis=1)
y = df['Appliances']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.to_csv("energydata_features_v1.csv", index=False)
y_train.to_csv("energydata_targets_v1.csv", index=False)
# !ls | grep energydata_
# !rm energydata_*.csv && echo "------------нет файлов"
# !ls | grep energydata_
# !dvc pull && echo "------------есть файлы"
# !ls | grep energydata_

In [ ]:
params = {'fit_intercept': [True,False]}
grid_search = GridSearchCV(estimator=lir(), param_grid=params, scoring='neg_mean_squared_error', cv=3,verbose=2, n_jobs=-1)
grid_search.fit(x_train[:100], y_train[:100])
print(f'best_params_ {grid_search.best_params_}')
print("predict",grid_search.predict(x_test)[0])
with open('model.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

### Конфигурируйм DVC для логирования метрик и гиперпараметров каждого эксперимента.


In [ ]:
%%sh
dvc exp remove -A
dvc queue remove --all
touch dvc.yaml
git checkout -b dvc-241106
dvc exp save --name first -m "241106"
dvc exp show

In [ ]:
import yaml
import json
import dvc.api

data = {"postprocessing":{"fit_intercept": True}}

with open("params.yaml", "w") as file:
    yaml.dump(data, file)

pipeline={
  "stages": {
    "process-data": {
      "cmd": "dvc run python script.py",
      "deps": [
        "energydata_features_v1.csv",
        "energydata_targets_v1.csv",
        "script.py"
      ],
      "outs": [
        "output/testing_data.pkl",
        "output/training_data.pkl"
      ],
      "params": [
        "fit_intercept"
      ]
    },
    "train": {
      "cmd": "pipenv run python script.py train",
      "deps": [
        "script.py",
        "output/training_data.pkl"
      ],
      "outs": [
        "output/model.pkl"
      ],
      "params": [
        "train.gamma"
      ]
    },
  }
}
with open("dvc.yaml", "w") as file:
    yaml.dump(pipeline, file)

params = dvc.api.params_show("params.yaml")
print(json.dumps(params, indent=4))

In [ ]:
%%sh
echo "import pickle\nimport pandas as pd\nwith open('model.pkl', 'rb') as f:\n    loaded_regr = pickle.load(f)\nfrom sklearn.linear_model import LinearRegression as lir\nlm=lir()\nprint('predict',loaded_regr.predict(pd.read_csv('/home/jovyan/notebooks/energydata_features_v1.csv'))[0])" >script.py
dvc stage list --names-only
dvc stage add --name=postprocessing --force python script.py

In [ ]:
!dvc experiments run --set-param +fit_intercept=False postprocessing

In [ ]:
!dvc exp show

### Сравниваем результаты нескольких экспериментов с помощью DVC


In [ ]:
!dvc experiments show

In [ ]:
!dvc metrics show

In [ ]:
!dvc metrics

In [ ]:
print(dvc.api.exp_show(),"\n",":"*80)
params = dvc.api.params_show()
print(params)
loaded_data=dvc.api.read('energydata_features_v1.csv')
print(loaded_data[:500])
print(    dvc.api.metrics_show())

In [ ]:
!pip install deepchecks --quiet

In [ ]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet -O /content/nyc_parking_violations_2022.parquet

In [ ]:
import pandas as pd

In [ ]:
# read 5 columns data:
df = pd.read_parquet(
    "/content/nyc_parking_violations_2022.parquet",
    columns=["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date", "Summons Number"]
)

# view a random sample of 10 rows:
df.sample(10)

In [ ]:
import numpy as np
import pandas as pd
import sklearn

import deepchecks

pd.set_option('display.max_columns', 45); SEED=832; np.random.seed(SEED);
from deepchecks.tabular.datasets.classification.phishing import load_data
#df = load_data(data_format='dataframe', as_train_test=False)
df.head(5)

In [ ]:
np.Inf = np.inf;np.NaN=np.nan #bug workaround
from deepchecks.tabular.suites import data_integrity

integ_suite = data_integrity()

dataset = deepchecks.tabular.Dataset(df=df, label='Summons Number',
                                     set_datetime_from_dataframe_index=True, cat_features=[])
integ_suite.run(dataset)

docker-compose.yaml
```
services:
  serverx:
    x-develop:
      watch:
        - action: sync+restart
          path: ./
          target: /home/user/app
        - action: rebuild
          path: ./requirements.txt
        - action: rebuild
          path: ./serverx_dockerfile
        - action: rebuild
          path: ./serverx-compose.yml

    build:
      context: .
      dockerfile: ./serverx_dockerfile
      target: runtime
    restart: always
    container_name: serverx
    image: serverx:2023-12-28
    hostname: inside_serverx
    ports:
      - "8080:8888"

    network_mode: host

    deploy:
      resources:
        limits:
          cpus: '0.50'
          memory: 50M
        reservations:
          cpus: '0.25'
          memory: 20M

    healthcheck:
      test: curl -sS http://127.0.0.1:8888/project/llm-providers || exit 1
      interval: 30s
      timeout: 60s
      retries: 3
      start_period: 30s
```

```
# syntax=docker/dockerfile-upstream:master
ARG VERSION="3.11"
FROM python:$VERSION as builderstage
RUN useradd -m -u 1000 user
USER user
ENV HOME=/home/user \
    PATH=/home/user/.local/bin:$PATH
COPY ./requirements.txt ./requirements.txt
RUN pip install -r requirements.txt

FROM python:$VERSION AS runtime
RUN /usr/bin/rm -f /etc/localtime && ln /usr/share/zoneinfo/Europe/Moscow  /etc/localtime
RUN useradd -m -u 1000 user
USER user
ENV HOME=/home/user \
    PATH=/home/user/.local/bin:$PATH
    # Make sure scripts in .local are usable:

COPY --from=builderstage /home/user/.local /home/user/.local
WORKDIR $HOME/app
COPY --chown=user . $HOME/app
RUN chown -R user:user $HOME/app
RUN chmod -R 600 $HOME/app

COPY . .
CMD ["===== server start ======="]
```

docker-compose.yaml
```
name: project_gitlabrunner
services:
  gitlabweb:
    image: 'gitlab/gitlab-ce:latest'
    restart: always
    hostname: '127.0.0.1'
    container_name: gitlabweb
    network_mode: host
    #dependon:
    #  - gitlabrunner
    ports:
      - '80:80'
      - '446:443'
      - '2224:22'
    volumes:
      - './config:/etc/gitlab'
      - './logs:/var/log/gitlab'
      - './data:/var/opt/gitlab'
    shm_size: '256m'
    secrets:
      - gitlab_root_password

  gitlabrunner:
    container_name: gitlabrunner_sd
    hostname: inside_gitlabrunner_sd
    image: gitlab/gitlab-runner:latest
    restart: always
    network_mode: host

    ports:
      - '8093:8093'
      - '8094:80'
    volumes:
      - './gitlab-runner-config:/etc/gitlab-runner'
      - '/var/run/docker.sock:/var/run/docker.sock'
    healthcheck:
      test: "gitlab-runner verify"
      interval: 120s
      timeout: 12s
      retries: 3
      start_period: 30s



secrets:
  gitlab_root_password:
    file: ./root_password.txt
```

.gitlab-ci
```
stages:
      - testing

    testing:
      stage: testing
      script:
        - echo 'Hello world'
    tags:
      - my-tags
```